In [1]:
import pandas as pd

# === 1. Cargar grilla WRF D03 (puntos sobre superficie) ===
grilla = pd.read_pickle("../0-data/grilla_WRF-D03_superficie.pkl")

# === 2. Calcular límites lat/lon con margen adicional (0.1°) ===
lat_min = grilla['lat'].min() - 0.1
lat_max = grilla['lat'].max() + 0.1
lon_min = grilla['lon'].min() - 0.1
lon_max = grilla['lon'].max() + 0.1

# === 3. Ajustar límites al múltiplo más cercano de 0.1° (resolución ERA5-Land) ===
lat_min = round(lat_min, 1)
lat_max = round(lat_max, 1)
lon_min = round(lon_min, 1)
lon_max = round(lon_max, 1)

# === 4. Guardar Bounding Box en formato [Norte, Oeste, Sur, Este] ===
bbox_era = [lat_max, lon_min, lat_min, lon_max]

print("Bounding Box ajustado a ERA5-Land (resolución 0.1°):")
print(f"[Norte, Oeste, Sur, Este] = {bbox_era}")


Bounding Box ajustado a ERA5-Land (resolución 0.1°):
[Norte, Oeste, Sur, Este] = [-32.4, -71.8, -33.6, -71.1]


In [2]:
import geopandas as gpd
import folium
from shapely.geometry import Point

# === 1. Crear puntos de la grilla ERA5-Land dentro del bounding box ===
lats = []
lons = []
lat_val = bbox_era[2]
while lat_val <= bbox_era[0]:
    lon_val = bbox_era[1]
    while lon_val <= bbox_era[3]:
        lats.append(round(lat_val, 6))
        lons.append(round(lon_val, 6))
        lon_val += 0.1
    lat_val += 0.1

era_points = gpd.GeoDataFrame(
    {"lat": lats, "lon": lons},
    geometry=[Point(xy) for xy in zip(lons, lats)],
    crs="EPSG:4326"
)

# === 2. Crear mapa centrado en la grilla WRF ===
centro = [grilla["lat"].mean(), grilla["lon"].mean()]
m = folium.Map(location=centro, zoom_start=8, tiles="CartoDB positron")

# === 3. Añadir bounding box en azul ===
folium.Rectangle(
    bounds=[[bbox_era[0], bbox_era[1]], [bbox_era[2], bbox_era[3]]],
    color="blue", fill=False, weight=2, tooltip="Bounding Box ERA5-Land"
).add_to(m)

# === 4. Añadir puntos WRF-D03 (rojo) ===
for _, row in grilla.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=2, color="red", fill=True, fill_opacity=0.7
    ).add_to(m)

# === 5. Añadir puntos ERA5-Land (verde) ===
for _, row in era_points.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=2, color="green", fill=True, fill_opacity=0.5
    ).add_to(m)

# === 6. Mostrar mapa interactivo ===
m


In [ ]:
import cdsapi
import os

# === 1. Inicializar cliente de descarga de Copernicus ===
c = cdsapi.Client()

# === 2. Carpeta de salida ===
out_dir = "era5land_daily_d03"  # Nombre del directorio
os.makedirs(out_dir, exist_ok=True)

# === 3. Bounding box (Norte, Oeste, Sur, Este) ===
# Ajustado a la malla de resolución 0.1° para cubrir el dominio D03
bbox_era = [-32.3, -71.9, -33.6, -71.0]

# === 4. Variables meteorológicas a descargar ===
variables = [
    "2m_dewpoint_temperature",
    "2m_temperature",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "surface_pressure",
    "total_precipitation"  
]


# === 5. Iterar por año y mes ===
for year in range(2014, 2020):  # Rango de años: 2014–2019
    for month in range(1, 13):  # Meses 1–12

        # Nombre del archivo de salida
        target_file = os.path.join(out_dir, f"era5land_daily_{year}_{month:02d}.nc")

        # Saltar si ya fue descargado
        if os.path.exists(target_file):
            print(f"{target_file} ya existe, saltando...")
            continue

        # Lista genérica de días (1 al 31) — el servidor ignora días inválidos
        days = [f"{d:02d}" for d in range(1, 32)]

        print(f"Descargando {year}-{month:02d}...")

        try:
            c.retrieve(
                "derived-era5-land-daily-statistics",
                {
                    "variable": variables,
                    "year": str(year),
                    "month": f"{month:02d}",
                    "day": days,
                    "daily_statistic": "daily_mean",
                    "time_zone": "utc+00:00",
                    "frequency": "1_hourly",
                    "area": bbox_era,
                    "data_format": "netcdf",
                    "download_format": "unarchived"
                },
                target_file
            )

        except Exception as e:
            print(f"Error en {year}-{month:02d}: {e}")
